# Import packages

In [1]:
import os
import json
import numpy as np
from sympy import symbols, sympify, lambdify
from tabulate import tabulate
import yaml
from datetime import datetime
import glob

import tensorflow as tf

tf.keras.backend.set_floatx('float64')

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import NonNeg

# Set random seeds for reproducibility
seed_value = 2023
tf.random.set_seed(seed_value)
np.random.seed(seed_value)

2023-11-15 11:28:27.241219: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-15 11:28:27.243269: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 11:28:27.289447: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 11:28:27.289507: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 11:28:27.289552: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

# Constants and parameters

In [2]:
tf.keras.backend.epsilon()

1e-07

In [3]:
tf_precision = tf.float64
np_precision = np.float64


with open('config_run.yml', 'r') as ymlfile:
    config_data = yaml.load(ymlfile, Loader=yaml.FullLoader)

In [4]:
problem_name = config_data['problem_folder']
EQS_FILE = problem_name + '.json'
EQS_PATH = os.path.join(os.getcwd(), problem_name, EQS_FILE)

training_steps = config_data['epochs']
display_step = training_steps // 10

load_initial_estimate = config_data['load_initial_estimate']
save_best_solution = config_data['save_best_solution']

# learning_rate = 1e-2
# learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay([training_steps // 2],[1e-2, 1e-3])
# learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay([training_steps // 3, 2 * training_steps // 3],[1e-1,1e-2,1e-3])
# learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay([training_steps // 3, 2 * training_steps // 3],[1e-2,5e-3,1e-3])
# learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay([training_steps // 3, 2 * training_steps // 3],[5e-2,1e-2,5e-3])

if len(list(config_data['learning_rate'].keys())) > 1:
    lr_steps = [int(training_steps * key / 100) for key in list(config_data['learning_rate'].keys()) if key != 0]
    lr_values = [float(val) for val in list(config_data['learning_rate'].values())]
    learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay(lr_steps, lr_values)
else:
    learning_rate = float(list(config_data['learning_rate'].values())[0])

# Load Equations

In [5]:
# Read the JSON data from the file
with open(EQS_PATH, 'r') as file:
    data = json.load(file)

variables = data['variables']
equations = data['equations']
# equation_terms = data['equation_terms']

variables_str = ' '.join(variables)
variables_sym = symbols(variables_str)
equations_sym = [sympify(eq) for eq in equations]  

## Identify the equation terms
equation_terms = []
for (i, eq) in enumerate(equations_sym):
    terms = []
    for term in eq.args:
        terms.append(str(term))
    equation_terms.append(terms)

# Initialize a list to store the coefficients
scalar_coefficients = []
functions = []
bias_coefficients = []

# Loop through the equation terms
for terms in equation_terms:
    bias_coefficients.append(float(terms[0]))
    coefficients = []   
    for term in terms[1:]:
        # Split each term by the first '*'
        term_parts = term.split('*')
        # Extract the scalar coefficient or default to '1'
        scalar_coeff = term_parts[0] if len(term_parts) > 1 and term_parts[0] not in variables else '1.0'
        coefficients.append(float(scalar_coeff))
               
        # Remove first occurrence of scalar coefficient from the term
        func_coeff = term.replace(term_parts[0]+'*', '', 1) if len(term_parts) > 1 and term_parts[0] not in variables else term
        functions.append(func_coeff)
        
    scalar_coefficients.append(coefficients)   
       
functions_sym = [sympify(func) for func in functions]       
        
print(bias_coefficients)
print(scalar_coefficients)
print(functions)
print(functions_sym)

[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
[[1.0, 1.0, 1.0, 1.0], [2.0, 2.0, 2.0], [6.0, 6.0], [3.0, 3.0, 3.0], [24.0], [12.0, 12.0], [8.0, 8.0], [4.0, 4.0, 4.0]]
['b_1', 'b_2', 'b_3', 'b_4', 'a_21*b_2', 'b_3*(a_31 + a_32)', 'b_4*(a_41 + a_42 + a_43)', 'b_4*(a_21*a_42 + a_43*(a_31 + a_32))', 'a_21*a_32*b_3', 'a_21**2*b_2', 'b_3*(a_31 + a_32)**2', 'b_4*(a_41 + a_42 + a_43)**2', 'a_21*a_32*a_43*b_4', 'b_4*(a_21**2*a_42 + a_43*(a_31 + a_32)**2)', 'a_21**2*a_32*b_3', 'b_4*(a_21*a_42 + a_43*(a_31 + a_32))*(a_41 + a_42 + a_43)', 'a_21*a_32*b_3*(a_31 + a_32)', 'a_21**3*b_2', 'b_3*(a_31 + a_32)**3', 'b_4*(a_41 + a_42 + a_43)**3']
[b_1, b_2, b_3, b_4, a_21*b_2, b_3*(a_31 + a_32), b_4*(a_41 + a_42 + a_43), b_4*(a_21*a_42 + a_43*(a_31 + a_32)), a_21*a_32*b_3, a_21**2*b_2, b_3*(a_31 + a_32)**2, b_4*(a_41 + a_42 + a_43)**2, a_21*a_32*a_43*b_4, b_4*(a_21**2*a_42 + a_43*(a_31 + a_32)**2), a_21**2*a_32*b_3, b_4*(a_21*a_42 + a_43*(a_31 + a_32))*(a_41 + a_42 + a_43), a_21*a_32*b_3*(a_31 + a_32), a

# Functions

In [6]:
def load_most_recent_estimate(problem_name, prefix="best_estimate"):
    # Find all JSON files with the specified prefix
    pattern = os.path.join(problem_name, f"{prefix}_*.json")
    files = glob.glob(pattern)

    if not files:
        print(f"No files found with prefix '{prefix}'")
        return None

    # Get the most recent file based on modification time
    most_recent_file = max(files, key=os.path.getmtime)
    
    print(f"Loading most recent estimate from {most_recent_file}")

    # Load json file with initial estimate (best from the most recent run)
    with open(most_recent_file, 'r') as file:
        data = json.load(file)

    return data['best_solution']



def evaluate_tf_function(inputs, values, symbolic_function):
    # Ensure that the number of inputs and values match
    if len(inputs) != len(values):
        raise ValueError("Number of inputs and values must match.")

    # Evaluate the symbolic function using TensorFlow and the provided values
    result = symbolic_function(*values)
    # Convert the result to a TensorFlow tensor
    result = tf.convert_to_tensor(result, dtype=tf_precision)
    
    # print(result.__class__)
    
    return result

# Define the custom activation function with @tf.function
# @tf.function
def activation_layer2(layer, vars=variables_sym, funcs=functions_sym):
    
    layer_values = layer.numpy()[0]
    var_vals = [tf.squeeze(layer[0, i]) for i in range(layer_values.shape[0])]
       
    layer_act = list()
    for func in funcs:
        func_tf = lambdify(vars, func, 'tensorflow')
        layer_act.append(evaluate_tf_function(variables_sym, var_vals, func_tf))
    
    # Convert layer_act to a TensorFlow tensor
    layer_2 = tf.convert_to_tensor(layer_act, dtype=tf_precision)
    layer_2 = tf.reshape(layer_2, [1, len(funcs)])

    return layer_2


# Create model
def multilayer_perceptron(weights, biases):

    # # Reshape input if necessary, matching the shape of the first layer's weights
    # x = tf.reshape(x, [1, -1])  # Adjust the shape as needed

    # layer_1 = tf.add(tf.matmul(x, weights['w12']), biases['b12'])
    # layer_1 = tf.matmul(x, weights['w12'])
    layer_1 = weights['w12']
    
    # print(f'layer_1 = {layer_1.shape}')

    layer_2 = activation_layer2(layer_1)
    
    # Output fully connected layer
    output = tf.add(tf.matmul(layer_2, weights['w34']), biases['out'])
    
    # # Add RELU activation function to output layer
    # output = tf.nn.relu(output)
    
    return output, layer_1


def loss_function(weights, biases, lambda_reg=0.01):
    
    output, _ = multilayer_perceptron(weights, biases)
    
    # # L2 regularization term for weights
    # regularization_term = sum(tf.nn.l2_loss(w) for w in weights.values())
    
    # L1 regularization term for weights - to enforce sparsity
    # L1 regularization adds the sum of the absolute values of the weights to the loss function. 
    # This can encourage the model to prefer solutions where many weights are exactly zero.
    regularization_term = sum(tf.reduce_sum(tf.abs(w)) for w in weights.values())

    # Mean squared error loss
    mse_loss = tf.reduce_mean(tf.square(output))

    # Total loss with regularization term
    total_loss = mse_loss + lambda_reg * regularization_term

    return total_loss


# Train step
def train_step(weights, biases, optimizer, lambda_reg=0.01):

    with tf.GradientTape() as tape:
        
        loss = loss_function(weights, biases, lambda_reg=lambda_reg)

    trainable_variables = [weights['w12']]  # list containing only 'w12'
    
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss    

# Model

## Create model

In [7]:
# Network Parameters
num_input = 1 # input layer

num_vars = len(variables)
num_eqs = len(equations)
num_functions = len(functions)

num_hidden = [num_vars, num_functions]
num_output = num_eqs # output layer

display(
    f'num_hidden = {num_hidden}',
    f'num_output = {num_output}'
)

'num_hidden = [10, 20]'

'num_output = 8'

In [8]:
## Weights to Layer 3 - determined by the functions contains the variables
w23_flags = [[True] * num_functions for _ in range(num_vars)]
for vi, var in enumerate(variables):
    for fi, func in enumerate(functions):
        # print(f'var = {var}, func = {func}', var not in func)
        if var not in func:
            w23_flags[vi][fi] = False

w23_flags = tf.constant(w23_flags, dtype=tf.bool)

# Initialize the weights (w23) with zeros
w23 = tf.constant(tf.zeros(num_hidden, dtype=tf_precision))
# Set the weights to 1 where func_flags is True
w23 = tf.where(w23_flags, tf.ones_like(w23), w23)
w23 = tf.transpose(w23)

## Weights to Layer 4 - determined by the coefficients of the functions
# Initialize the weights (w34) with zeros
w34_np = np.zeros([num_functions, num_output], dtype=np_precision)
# Assign scalar coefficients to the first column of w34_np
row = 0
for sci, scalar_coeffs in enumerate(scalar_coefficients):
    for scj, scl_coeff in enumerate(scalar_coeffs):
        w34_np[row, sci] = scl_coeff
        row += 1
# Tranform to tensor
w34 = tf.constant(w34_np, dtype=tf_precision)

if load_initial_estimate is None:
    w12 = tf.Variable(tf.random.normal([num_input, num_hidden[0]]), 
                      dtype=tf_precision, 
                      constraint=NonNeg())
    
    initial_solution = w12.numpy()[0].tolist()
    
elif isinstance(load_initial_estimate, str):
    print('Loading initial estimate...')
    # Load json file with initial estimate (best from the previous run)
    with open(os.path.join(problem_name, f'{load_initial_estimate}.json'), 'r') as file:
        data = json.load(file)
    init_estimate = data['best_solution']
    initial_solution = [float(x) for x in init_estimate.values()]
    w12 = tf.Variable(tf.constant([initial_solution], dtype=tf_precision), constraint=NonNeg())

elif load_initial_estimate == True:
    init_estimate = load_most_recent_estimate(problem_name)
    initial_solution = [float(x) for x in init_estimate.values()]
    w12 = tf.Variable(tf.constant([initial_solution], dtype=tf_precision), constraint=NonNeg())
    
# Store layers weight & bias
weights = {
    ## Variables - weights to Layer 1
    # Create the variable with the non-negativity constraint
    'w12': w12,
    # Whether the functions in F1 and F2 contain the variables x1 and x2
    'w23': w23,
    # # The coefficients of the functions in F1 and F2
    'w34': w34
}

biases = {
    'b12': tf.constant(tf.zeros([num_hidden[0]], dtype=tf_precision)),
    'b23': tf.constant(tf.zeros([num_hidden[1]], dtype=tf_precision)),
    'out': tf.constant([bias_coefficients], dtype=tf_precision),
}

# Stochastic gradient descent optimizer.
optimizer = Adam(learning_rate=learning_rate, name='custom_optimizer_name')

Loading most recent estimate from ERK_equations_s4p4/best_estimate_231115_111206.json


In [9]:
weights['w12']

<tf.Variable 'Variable:0' shape=(1, 10) dtype=float64, numpy=
array([[5.00000017e-01, 2.46398870e-08, 4.99999964e-01, 3.64818036e-08,
        3.19337196e-07, 9.99999505e-01, 1.66666715e-01, 3.33333288e-01,
        3.33333246e-01, 1.66666762e-01]])>

## Train model

In [10]:
best_loss = np.inf  # Initialize with infinity or an appropriate initial value

# Initialize variables to store the best solution
best_solution = None
best_epoch = None
# # Define the number of epochs after which to save the best weights
# save_interval = training_steps // 100    # Every 1% of the training steps 

lambda_reg = float(config_data['lambda_regularization'])

for i in range(training_steps):
    
    current_loss = train_step(weights, biases, optimizer, lambda_reg=lambda_reg)
    
    # Check if the current loss is better than the best loss
    if current_loss < best_loss:
        best_loss = current_loss
        best_epoch = i
        best_solution = weights['w12'].numpy()[0].copy()
       
    if i % display_step == 0:
        print(f"epoch {i:6d} => loss: {current_loss:.16e} | best loss: {best_loss:.16e} ({best_epoch})")
        
    if current_loss <= 1e-20:
        print(f"epoch {i:6d} => loss: {current_loss:.16e} | best loss: {best_loss:.16e} ({best_epoch})")
        break

epoch      0 => loss: 1.6882528101164943e-15 | best loss: 1.6882528101164943e-15 (0)
epoch    500 => loss: 5.7673191535376276e-17 | best loss: 4.3841518673133084e-17 (363)
epoch   1000 => loss: 9.4040595952352286e-14 | best loss: 3.6554897158223421e-17 (948)
epoch   1500 => loss: 1.4377210317776170e-13 | best loss: 2.6449987964127928e-17 (1459)
epoch   2000 => loss: 9.4580934370895463e-17 | best loss: 1.7974035556307145e-17 (1777)
epoch   2500 => loss: 2.6552143548175160e-15 | best loss: 1.2756454768679111e-17 (2002)
epoch   3000 => loss: 1.0109628541630270e-17 | best loss: 1.0109628541630270e-17 (3000)
epoch   3500 => loss: 8.6772750393882910e-18 | best loss: 8.6772750393882910e-18 (3500)
epoch   4000 => loss: 7.2034617527009997e-18 | best loss: 7.2034617527009997e-18 (4000)
epoch   4500 => loss: 5.7613096491868656e-18 | best loss: 5.7613096491868656e-18 (4500)
epoch   5000 => loss: 4.4241601425007948e-18 | best loss: 4.4241601425007948e-18 (5000)


## Save best solution

In [11]:
if save_best_solution is not None:
    
    best_solution_dict = dict()
    initial_solution_dict = dict()
    for vi, var in enumerate(variables):
        initial_solution_dict[var] = f'{initial_solution[vi]:.16f}'
        best_solution_dict[var] = f'{best_solution[vi]:.16f}'
        
    save_solution_dict = {
        'initial_estimate': initial_solution_dict,
        'best_solution': best_solution_dict,
        'best_epoch': best_epoch,
    }
        
    now = datetime.now()
    dt_string = now.strftime("%y%m%d %H%M%S")
    save_file_path = os.path.join(problem_name, f'best_estimate_{dt_string.split()[0]}_{dt_string.split()[1]}.json')
    
    # Save the configuration data to the JSON file
    with open(save_file_path, 'w') as config_file:
        json.dump(save_solution_dict, config_file, indent=4)

In [12]:
func_res_model, solution  = multilayer_perceptron(weights, biases)
solution = list(solution.numpy()[0])
best_solution = list(best_solution)

print('BEST SOLUTION')
for i, var in enumerate(variables):
    print(f'{var} = {best_solution[i]:.16f}')
    
print()
print('RESIDUALS')
# headers = ['Equation', 'Model residual', 'Function residual']
headers = ['Equation', 'Function residual']
table_data = []

for f, func in enumerate(equations):
    eq = equations_sym[f]
    equation = lambdify(variables_sym, eq)
    func_res_eq = abs(equation(*best_solution))
    # table_data.append([f'{func} = ', f'{abs(func_res_model[0, f]):.4e}', f'{func_res_eq:.4e}'])
    table_data.append([f'{func} = ', f'{func_res_eq:.4e}'])

print(tabulate(table_data, headers=headers, tablefmt='grid'))

BEST SOLUTION
a_21 = 0.4999999987634647
a_31 = 0.0000000165469802
a_32 = 0.4999999829570089
a_41 = 0.0000000312536184
a_42 = 0.0000000515630700
a_43 = 0.9999998729895415
b_1 = 0.1666666807711204
b_2 = 0.3333333081783182
b_3 = 0.3333333171689756
b_4 = 0.1666666938983497

RESIDUALS
+---------------------------------------------------------------------------------------------------+---------------------+
| Equation                                                                                          |   Function residual |
+===================================================================================================+=====================+
| b_1 + b_2 + b_3 + b_4 - 1 =                                                                       |          1.6764e-11 |
+---------------------------------------------------------------------------------------------------+---------------------+
| 2*a_21*b_2 + 2*b_3*(a_31 + a_32) + 2*b_4*(a_41 + a_42 + a_43) - 1 =                              